Run 17 July


*   Given an input file of passages = phase_2_passages.jsonl
*   Generate 3 questions per passage.


*   Model used: Meta-LLaMA-3-8B-Instruct, 8B
*   Note: Do not run phases 1, and 2.





# File Mounting and Setup

In [1]:
# --- Installation Block ---
# Install/upgrade necessary libraries for the entire pipeline.
# It's crucial to run these installations first.

# For extracting text from PDF files (if needed in earlier steps)
!pip install pdfplumber

# For working with sentence embeddings (if needed for other phases like RAG)
!pip install sentence-transformers

# Core libraries for Large Language Models (LLMs) from Hugging Face
# `transformers`: The main library for model loading, tokenization, and generation (for Llama 3).
# `accelerate`: Helps with efficient model loading and inference, especially on GPUs.
# `bitsandbytes`: Essential for 4-bit quantization, which significantly reduces memory usage and speeds up inference.
!pip install --upgrade transformers
!pip install accelerate
!pip install bitsandbytes

# --- Import Block ---
# Import all necessary Python modules and components from installed libraries.
# These imports will be available after the runtime has been restarted following installations.

import os        # For interacting with the operating system (e.g., creating directories, joining paths)
import json      # For working with JSON data (reading .jsonl files)
import torch     # PyTorch library, fundamental for deep learning models and GPU operations
from tqdm import tqdm # For displaying progress bars during long loops (e.g., passage processing)
import re        # For regular expressions, used to parse and clean generated questions

# Specific imports from the transformers library for LLM operations
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print("Libraries installed/upgraded and imports completed.")
print("IMPORTANT: Restart the Colab runtime now (Runtime > Disconnect and delete runtime).")
print("After restarting, run this block again, and then proceed to the next blocks.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [2]:
def free_memory():
    import gc, torch
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print(" Memory cleaned (RAM + GPU cache).")

In [6]:
# UNCOMMENT ONLY IF NEED TO UPLOAD NEW FILES
# from google.colab import files
# import shutil

# uploaded = files.upload()  # Manually select geetha_vahini.pdf
# filename = next(iter(uploaded))  # Get the uploaded filename

# # Move uploaded file to persistent storage
# shutil.move(filename, os.path.join(persistent_dir, filename))


In [3]:
import os
from google.colab import drive

"""
Checks for the existence of specific files within a predefined Google Drive path.
Mounts Google Drive if not already mounted.
"""
print("Mounting Google Drive...")
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure you are running this in a Google Colab environment.")


# Updated DATA_PATH to include the 'geetha_vahini' subdirectory
DATA_PATH = "/content/drive/MyDrive/fpdata/geetha_vahini/"

FILE_NAME_ORIGINAL_PDF = "geetha_vahini.pdf"
FILE_NAME_CLEAN_TEXT = "phase_1_clean.txt"
FILE_NAME_PASSAGES = "phase_2_passages.jsonl"
FILE_NAME_QUESTIONS = "phase_3_questions_v3_gemma-n.jsonl"

full_path_original_pdf = os.path.join(DATA_PATH, FILE_NAME_ORIGINAL_PDF)
full_path_clean_text = os.path.join(DATA_PATH, FILE_NAME_CLEAN_TEXT)
full_path_passages = os.path.join(DATA_PATH, FILE_NAME_PASSAGES)
full_path_questions = os.path.join(DATA_PATH, FILE_NAME_QUESTIONS)

print(f"\nChecking accessibility of files in Google Drive at: {DATA_PATH}\n")

if os.path.exists(full_path_original_pdf):
    print(f"'{FILE_NAME_ORIGINAL_PDF}' found at: {full_path_original_pdf}")
else:
    print(f"'{FILE_NAME_ORIGINAL_PDF}' NOT found at: {full_path_original_pdf}")
    print(f"Please ensure '{FILE_NAME_ORIGINAL_PDF}' is in the '{DATA_PATH}' folder in your Google Drive.")

print("-" * 50)

if os.path.exists(full_path_clean_text):
    print(f"'{FILE_NAME_CLEAN_TEXT}' found at: {full_path_clean_text}")
else:
    print(f"'{FILE_NAME_CLEAN_TEXT}' NOT found at: {full_path_clean_text}")
    print(f"Please ensure '{FILE_NAME_CLEAN_TEXT}' is in the '{DATA_PATH}' folder in your Google Drive.")

print("-" * 50)

if os.path.exists(full_path_passages):
    print(f"'{FILE_NAME_PASSAGES}' found at: {full_path_passages}")
else:
    print(f"'{FILE_NAME_PASSAGES}' NOT found at: {full_path_passages}")
    print(f"Please ensure '{FILE_NAME_PASSAGES}' is in the '{DATA_PATH}' folder in your Google Drive.")

print("-" * 50)

if os.path.exists(full_path_questions):
    print(f"'{FILE_NAME_QUESTIONS}' found at: {full_path_questions}")
else:
    print(f"'{FILE_NAME_QUESTIONS}' NOT found at: {full_path_questions}")
    print(f"Please ensure '{FILE_NAME_QUESTIONS}' is in the '{DATA_PATH}' folder in your Google Drive.")

print("\nIf files are still not found after mounting, double-check the folder path and file names in your Google Drive.")
print("You can also use the file browser icon on the left-hand side of Colab to navigate your mounted Drive and verify paths.")

# To use the function, simply call it:
# check_files_exist_on_google_drive()


Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully!

Checking accessibility of files in Google Drive at: /content/drive/MyDrive/fpdata/geetha_vahini/

'geetha_vahini.pdf' found at: /content/drive/MyDrive/fpdata/geetha_vahini/geetha_vahini.pdf
--------------------------------------------------
'phase_1_clean.txt' found at: /content/drive/MyDrive/fpdata/geetha_vahini/phase_1_clean.txt
--------------------------------------------------
'phase_2_passages.jsonl' found at: /content/drive/MyDrive/fpdata/geetha_vahini/phase_2_passages.jsonl
--------------------------------------------------
'phase_3_questions_v3_gemma-n.jsonl' found at: /content/drive/MyDrive/fpdata/geetha_vahini/phase_3_questions_v3_gemma-n.jsonl

If files are still not found after mounting, double-check the folder path and file names in your Google Drive.
You can also use the file browser icon on the left-hand side of Colab to navigate your mounted Drive and verify paths.


# Phase 1: Extract Clean Text

    # Phase 1: extract_clean_text(pdf_path: str)

        outputs/
        └── ABCDEF_vahini/
            └── phase_1_clean.txt

In [ ]:
import os
import pdfplumber

def extract_clean_text(pdf_path: str):
    """
    Phase 1: Extracts clean text from the given PDF and saves it in persistent storage.

    Args:
        pdf_path (str): Full path to input PDF (e.g., "/content/drive/MyDrive/fpdata/geetha_vahini.pdf")

    Output:
        Saves clean text to <DATA_PATH>/<base_name>/phase_1_clean.txt
    """
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_dir = os.path.join(DATA_PATH, base_name)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "phase_1_clean.txt")

    print(f"[Phase 1] Extracting text from: {pdf_path}")
    print(f"[Phase 1] Saving to: {output_path}")

    with pdfplumber.open(pdf_path) as pdf, open(output_path, "w", encoding="utf-8") as f_out:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                clean_text = text.strip().replace("\n", " ")
                f_out.write(clean_text + "\n")

    print(f"[Phase 1] Extraction complete.")
    return output_path


In [ ]:
# TEST PHASE 1
clean_text_file_path = extract_clean_text(pdf_path)

[Phase 1] Extracting text from: /content/drive/MyDrive/fpdata/geetha_vahini.pdf
[Phase 1] Saving to: /content/drive/MyDrive/fpdata/geetha_vahini/phase_1_clean.txt
[Phase 1] Extraction complete.


# Phase 2: Split the clean text into 200-word chunks (i.e. create passages)

    # PHASE 2: Split the clean text into 200-word chunks with ∼20-word overlap using NLTK

        outputs/
        └── ABCDEF_vahini/
            ├── phase_1_clean.txt
            └── phase_2_passages.jsonl


In [ ]:
import os
import json
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

# Ensure NLTK models are downloaded
nltk.download("punkt")

def create_passages_from_clean_text(clean_txt_path: str, chunk_size=200, overlap=20):
    """
    Phase 2: Splits clean text into overlapping chunks and saves to JSONL in persistent storage.

    Args:
        clean_txt_path (str): Full path to Phase 1 output (clean .txt file)
        chunk_size (int): Number of words per chunk
        overlap (int): Number of overlapping words between chunks

    Output:
        Saves JSONL chunks to <DATA_PATH>/<base_name>/phase_2_passages.jsonl
    """
    base_name = os.path.basename(os.path.dirname(clean_txt_path))
    output_dir = os.path.join(DATA_PATH, base_name)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "phase_2_passages.jsonl")

    with open(clean_txt_path, "r", encoding="utf-8") as f:
        text = f.read()

    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sent in sentences:
        tokens = word_tokenize(sent)
        if current_length + len(tokens) > chunk_size:
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)
            current_chunk = current_chunk[-overlap:]  # retain overlap
            current_length = sum(len(word_tokenize(s)) for s in current_chunk)
        current_chunk.append(sent)
        current_length += len(tokens)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    with open(output_path, "w", encoding="utf-8") as f_out:
        for i, chunk in enumerate(chunks):
            json.dump({"doc_id": f"{base_name}_{i:04d}", "text": chunk}, f_out)
            f_out.write("\n")

    print(f"[Phase 2] Chunked text saved to: {output_path}")
    return output_path


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# TEST PHASE 2
created_passages_file_path = create_passages_from_clean_text(clean_text_file_path, chunk_size=200, overlap=20)

[Phase 2] Chunked text saved to: /content/drive/MyDrive/fpdata/geetha_vahini/phase_2_passages.jsonl


In [ ]:
free_memory()

 Memory cleaned (RAM + GPU cache).


# Phase 3 - Generate Questions.
3 Questions per passage, using llama 8B

    Phase 3: Generate questions for each passage using a generative model (t5-base-qg-hl)
    Phase 3: generate_questions_for_passages(passages_jsonl_path: str, num_questions=3)    
    outputs/
    └── ABCDEF_vahini/
        ├── phase_1_clean.txt
        ├── phase_2_chunks.jsonl
        └── phase_3_questions.jsonl


In [4]:
# Global configuration: Define DATA_PATH where your input passages are and output will be saved.
# This path should point to the 'geetha_vahini' subdirectory within your 'fpdata' folder on Google Drive.
persistent_dir = "/content/drive/MyDrive/fpdata/geetha_vahini"
DATA_PATH = persistent_dir

def generate_questions_for_passages(passages_file_jsonl_path: str, num_questions=3, batch_size=8):
    """
    Phase 3: Generate diverse questions per passage using Meta-LLaMA-3-8B-Instruct model with optimizations.

    Args:
        passages_file_jsonl_path (str): Path to Phase 2 passage JSONL file (e.g., "phase_2_passages.jsonl").
        num_questions (int): Number of diverse questions to generate per passage.
        batch_size (int): Number of passages to process simultaneously. Adjust based on available VRAM
                          and desired speed. Larger batches generally mean faster processing up to a point.

    Output:
        Saves a JSONL file to <DATA_PATH>/phase_3_questions_v3_llama-3-8b.jsonl,
        containing the generated questions for each passage.
    """
    # Ensure the output directory exists. In this setup, it's the same as DATA_PATH.
    output_dir = DATA_PATH
    os.makedirs(output_dir, exist_ok=True)

    # Define the full path for the output file where generated questions will be permanently saved.
    output_path = os.path.join(output_dir, "phase_3_questions_v3_llama-3-8b.jsonl")

    # Define the specific Llama 3 Instruct model to be used from Hugging Face.
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

    # Load the tokenizer for the specified model.
    # `trust_remote_code=True` is often required for Llama models from Hugging Face.
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    # OPTIMIZATION 1: 4-bit Quantization Configuration
    # This configuration tells `bitsandbytes` how to load the model in 4-bit precision.
    # `bnb_4bit_compute_dtype=torch.float16` is used to perform computations in float16
    # even though weights are stored in 4-bit, which helps with numerical stability.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,                 # Enables 4-bit loading
        bnb_4bit_quant_type="nf4",         # Specifies the NormalFloat4 quantization type
        bnb_4bit_compute_dtype=torch.float16, # Sets the data type for 4-bit computations
    )

    print(f"Loading model {model_name} with 4-bit quantization...")
    # Load the model using the defined quantization configuration and automatic device mapping.
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,    # Applies the 4-bit quantization
        device_map='auto',                 # Automatically maps model layers to available GPUs
        trust_remote_code=True             # Required for Llama 3 models
    )
    print("Model loaded successfully.")

    # Determine the device (GPU 'cuda' or CPU) where input tensors will be placed.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Configure tokenizer padding for efficient batch processing with causal LMs.
    # Llama 3 models typically use the EOS token as a pad token.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left" # Left padding is generally preferred for causal models

    def generate_questions_batch(texts_batch):
        """
        Generates questions for a batch of texts using Llama 3's specific instruction format.
        """
        prompts = []
        for text in texts_batch:
            # Construct the prompt using Llama 3's official chat template.
            # This is crucial for getting high-quality responses from Llama 3 Instruct models.
            messages = [
                {"role": "system", "content": "You are a helpful assistant that generates questions based on provided passages."},
                {"role": "user", "content": f"Passage: {text}\nGenerate {num_questions} questions based on the passage:\n1."}
            ]
            # Apply the chat template to convert the list of messages into a single, formatted prompt string.
            # `tokenize=False` means we get the string, `add_generation_prompt=True` adds the final assistant turn start.
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            prompts.append(prompt) # OPTIMIZATION 2: Part 1 - Prompts for the entire batch are prepared here.

        # Tokenize the entire batch of prompts simultaneously.
        # `padding=True` ensures all sequences in the batch are padded to the same length.
        # `truncation=True` handles passages longer than `max_length`.
        # `max_length=512` is sufficient for 200-word passages (approx. 250-300 tokens).
        inputs = tokenizer(
            prompts, # OPTIMIZATION 2: Part 2 - The batch of prompts is tokenized together.
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        # Estimate the maximum number of new tokens the model should generate.
        # This prevents excessively long or irrelevant generations.
        max_new_tokens_per_question_estimate = 30
        max_new_tokens = num_questions * max_new_tokens_per_question_estimate + 10 # Add a small buffer

        # Generate the output for the entire batch.
        outputs = model.generate(
            **inputs, # OPTIMIZATION 2: Part 3 - The entire batch of inputs is passed to the model for parallel generation.
            max_new_tokens=max_new_tokens,
            do_sample=True,                # Enables sampling for more diverse questions.
            top_k=50,                      # Samples from the top 50 most likely tokens.
            top_p=0.95,                    # Uses nucleus sampling (tokens summing to 95% probability).
            temperature=0.7,               # Controls randomness; lower values make output more deterministic.
            num_return_sequences=1,        # Generates one complete sequence of questions per input prompt.
            pad_token_id=tokenizer.pad_token_id # Required for proper padding during generation.
        )

        # Decode the generated token IDs back into human-readable text for the entire batch.
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_questions_for_batch = []

        for i, generated_text in enumerate(decoded_outputs):
            # Llama 3's chat template includes specific tags. We need to extract only the assistant's response.
            assistant_start_tag = "<|start_header_id|>assistant<|end_header_id|>\n"
            start_index = generated_text.find(assistant_start_tag)
            if start_index != -1:
                generated_text = generated_text[start_index + len(assistant_start_tag):].strip()
            # Remove any trailing End-Of-Turn token if the model generated it.
            generated_text = generated_text.replace("<|eot_id|>", "").strip()

            # Use a regular expression to parse the generated questions, assuming a numbered list format.
            questions = re.findall(r'\d+\.\s*(.*?)(?=\n\d+\.|\n*$)', generated_text, re.DOTALL)
            cleaned_questions = [q.strip() for q in questions if q.strip()]

            # Ensure exactly `num_questions` are returned. Fill with placeholders if fewer are generated.
            while len(cleaned_questions) < num_questions:
                cleaned_questions.append(f"Generated question {len(cleaned_questions) + 1} (failed to generate distinct question).")
            all_questions_for_batch.append(cleaned_questions[:num_questions])

        return all_questions_for_batch

    # Count the total number of passages for the progress bar.
    with open(passages_file_jsonl_path, "r", encoding="utf-8") as f:
        total_passages = sum(1 for _ in f)

    # Initialize buffers to hold passages and their IDs before processing in batches.
    passages_buffer = [] # OPTIMIZATION 2: Part 4 - Buffer to collect passages for batching.
    doc_ids_buffer = []

    print(f"Starting question generation for {total_passages} passages with batch size {batch_size}...")
    # Open input and output files. The output file is opened in write mode, creating it if it doesn't exist.
    with open(passages_file_jsonl_path, "r", encoding="utf-8") as f_in, \
            open(output_path, "w", encoding="utf-8") as f_out:
        # Iterate through passages, adding them to the buffer.
        for line in tqdm(f_in, total=total_passages, desc="Generating Questions"):
            item = json.loads(line)
            passages_buffer.append(item["text"])
            doc_ids_buffer.append(item["doc_id"])

            # If the buffer is full (i.e., contains a complete batch), process it.
            if len(passages_buffer) == batch_size: # OPTIMIZATION 2: Part 5 - Triggering batch processing when buffer is full.
                batch_generated_questions = generate_questions_batch(passages_buffer) # OPTIMIZATION 2: Part 6 - Calling the batch generation function.
                # Write the results of the processed batch to the output file.
                for i in range(len(passages_buffer)):
                    json.dump({"doc_id": doc_ids_buffer[i], "queries": batch_generated_questions[i]}, f_out)
                    f_out.write("\n") # Add a newline for JSONL format.
                # Clear buffers for the next batch.
                passages_buffer = []
                doc_ids_buffer = []

        # Process any remaining passages in the last (possibly incomplete) batch.
        if passages_buffer:
            batch_generated_questions = generate_questions_batch(passages_buffer)
            for i in range(len(passages_buffer)):
                json.dump({"doc_id": doc_ids_buffer[i], "queries": batch_generated_questions[i]}, f_out)
                f_out.write("\n")

    print(f"[Phase 3] Questions saved to: {output_path}")
    return output_path


In [5]:
full_path_passages

'/content/drive/MyDrive/fpdata/geetha_vahini/phase_2_passages.jsonl'

In [6]:
print("Starting question generation...")

# Call the generate_questions_for_passages function.
# You can adjust the `batch_size` here. Start with 8 or 16 and increase if your A100 has more VRAM or can handle it.
# The output will be saved permanently to the specified output_path within the function.
output_file_path = generate_questions_for_passages(
    passages_file_jsonl_path=full_path_passages,
    num_questions=3,
    batch_size=8 # Adjust this value (e.g., 16, 32) to optimize speed vs. VRAM usage
)

print(f"\nQuestion generation process completed. Output saved permanently to: {output_file_path}")

Starting question generation...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Loading model meta-llama/Meta-Llama-3-8B-Instruct with 4-bit quantization...


...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Model loaded successfully.
Starting question generation for 4721 passages with batch size 8...


Generating Questions: 100%|██████████| 4721/4721 [1:29:31<00:00,  1.14s/it]


[Phase 3] Questions saved to: /content/drive/MyDrive/fpdata/geetha_vahini/phase_3_questions_v3_llama-3-8b.jsonl

Question generation process completed. Output saved permanently to: /content/drive/MyDrive/fpdata/geetha_vahini/phase_3_questions_v3_llama-3-8b.jsonl
